In [1]:
from physhapes.helper_functions import *
from physhapes.mcmc import * 

import numpy as np


In [2]:
# MCMC settings
num_chains = 3
num_samples = 3000
dt = 0.05
lambd = 0.9
obs_var = 0.0001
rb = 1
prior_sigma_min = 0.0
prior_sigma_max = 1.0
prior_alpha_min = 0.0
prior_alpha_max = 0.03
proposal_sigma_tau = 0.1
proposal_alpha_tau = 0.01
#seed_start = np.random.randint(0,1000_000_000)
n = 20
d = 2

# high level names 
experiment_path = "sigma=0.4_alpha=0.025_dt=0.05/seed=3822439040"
datapath = experiment_path + "/procrustes_aligned_rotated45.csv"
phylo_path = "../data/chazot_subtree_rounded.nw"
sub_id = str(np.random.randint(0, 1000_000_000))

In [3]:
#with open (f'{sub_id}.sh', 'w') as rsh:
#    rsh.write(f'''#!/bin/bash

#for i in $(seq 1 {num_chains}); do
#  screen -md -S {sub_id}_chain=$i python run_mcmc.py --outputpath {experiment_path}/mcmc/id={sub_id} --phylopath {phylo_path} --datapath {datapath} --prior_sigma_min {prior_sigma_min} --prior_sigma_max {prior_sigma_max} --prior_alpha_min {prior_alpha_min} --prior_alpha_max {prior_alpha_max} --proposal_sigma_tau {proposal_sigma_tau} --proposal_alpha_tau {proposal_alpha_tau} --rb {rb} --obs_var {obs_var} --lambd {lambd} --dt {dt} --n {n} --d {d} --N {num_samples} --use_wandb True --wandb_project "SPMS_MCMC"
#done
#'''
#    )

In [4]:
def create_mcmc_screen_script(experiment_path=None, datapath=None, phylo_path=None, num_chains=3, 
                              num_samples=3000, dt=0.05, lambd=0.7, obs_var=0.001, rb=2,
                              prior_sigma_min=0.0, prior_sigma_max=2.5,
                              prior_alpha_min=0.0, prior_alpha_max=0.03,
                              proposal_sigma_tau=0.2, proposal_alpha_tau=0.005,
                              n=20, d=2):
    """Creates a bash script to run MCMC chains in separate screen sessions."""
    
    script_name = f'screen_{sub_id}.sh'
    output_path = f"{experiment_path}/mcmc/id={sub_id}"
    
    with open(script_name, 'w') as rsh:
        rsh.write(f'''#!/bin/bash
# Make output directory
mkdir -p {output_path}
for i in $(seq 1 {num_chains}); do
  screen -md -S {sub_id}_chain=$i python run_mcmc.py --outputpath {output_path} --phylopath {phylo_path} --datapath {datapath} --prior_sigma_min {prior_sigma_min} --prior_sigma_max {prior_sigma_max} --prior_alpha_min {prior_alpha_min} --prior_alpha_max {prior_alpha_max} --proposal_sigma_tau {proposal_sigma_tau} --proposal_alpha_tau {proposal_alpha_tau} --rb {rb} --obs_var {obs_var} --lambd {lambd} --dt {dt} --n {n} --d {d} --N {num_samples} --use_wandb True --wandb_project "SPMS_MCMC"
done
''')
    
    # Make script executable
    import os
    os.chmod(script_name, 0o755)
    
    print(f"Created MCMC screen script: {script_name}")
    return script_name

In [5]:
script = create_mcmc_screen_script(
    experiment_path=experiment_path,
    datapath=datapath,
    phylo_path=phylo_path,
    num_chains=num_chains,
    num_samples=num_samples,
    dt=dt,
    lambd=lambd,
    obs_var=obs_var,
    rb=rb,
    prior_sigma_min=prior_sigma_min,
    prior_sigma_max=prior_sigma_max,
    prior_alpha_min=prior_alpha_min,
    prior_alpha_max=prior_alpha_max,
    proposal_sigma_tau=proposal_sigma_tau,
    proposal_alpha_tau=proposal_alpha_tau,
    n=n,
    d=d
)

print(f"bash {script}")

Created MCMC screen script: screen_536755528.sh
bash screen_536755528.sh
